In [91]:
import boto3

In [92]:
import pandas as pd
from io import StringIO

In [93]:
# Set up S3 client
client = boto3.client('s3')

In [94]:
folders_list =  ['forecast_7d_25_11_2022', 'forecast_7d_20_12_2022', 'forecast_7d_29_12_2022', 
                'forecast_7d_20_03_2023', 'forecast_7d_05_05_2023', 'forecast_7d_30_06_2023', 'forecast_7d_10_09_2023']

In [95]:
# Bucket name and common prefix
bucket_forecast = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
common_path = 'Forecast/Forecast_analysis/'

In [96]:
# Initialize a list to store DataFrames
dfs = []

In [97]:
# Iterate over each folder in the list
for folder_item in folders_list:
    # Build the new path for the current folder
    path = f'{common_path}{folder_item}/'
    
    # List objects in the folder
    elements = client.list_objects(Bucket=bucket_forecast, Prefix=path)

    # Iterate over each object in the folder
    for obj in elements.get('Contents', []):
        # Check if the object is a CSV file
        if obj['Key'].endswith('.csv'):
            # Read CSV content from S3
            response = client.get_object(Bucket=bucket_forecast, Key=obj['Key'])
            csv_content = response['Body'].read().decode('utf-8')

            # Transform CSV content to DataFrame
            df = pd.read_csv(StringIO(csv_content))

            
            df['folder_id'] = folder_item # 'folder_id' column to indentify the forecast

            dfs.append(df)

In [98]:
# Concatenate all DataFrames into a single one
final_df = pd.concat(dfs, ignore_index=True)

/tmp/ipykernel_19/3299088048.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(dfs, ignore_index=True)


In [99]:
final_df['folder_id'].unique()

array(['forecast_7d_25_11_2022', 'forecast_7d_20_12_2022',
       'forecast_7d_29_12_2022', 'forecast_7d_20_03_2023',
       'forecast_7d_05_05_2023', 'forecast_7d_30_06_2023',
       'forecast_7d_10_09_2023'], dtype=object)

In [100]:
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

In [101]:
final_df.drop('item_id', axis=1, inplace=True)

In [102]:
final_df = final_df[['folder_id', 'date','country', 'payer', 'p10', 'p50', 'p90', 'mean']]

In [103]:
final_df['date'].nunique()

49

In [104]:
final_df = final_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

/tmp/ipykernel_19/2399063668.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df = final_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [105]:
final_df.head()

,folder_id,date,country,payer,p10,p50,p90,mean
0,FORECAST_7D_25_11_2022,2022-11-26,ECUADOR,ECUAGIROS,50959,67213,82265,67214
1,FORECAST_7D_25_11_2022,2022-11-27,ECUADOR,ECUAGIROS,52365,66641,83326,66882
2,FORECAST_7D_25_11_2022,2022-11-28,ECUADOR,ECUAGIROS,68499,88875,109519,89118
3,FORECAST_7D_25_11_2022,2022-11-29,ECUADOR,ECUAGIROS,55936,72019,89362,72211
4,FORECAST_7D_25_11_2022,2022-11-30,ECUADOR,ECUAGIROS,47200,63398,78582,63426


In [106]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [107]:
#pip install pyathena

In [108]:
from pyathena import connect

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df_daily_check = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_19/268183201.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_daily_check = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [109]:
df_daily_check['date'] = pd.to_datetime(df_daily_check['date']).dt.date

In [110]:
df_daily_check.head()

,payer,country,date,tx,amount,coupon_count
0,BANCO AGRICOLA,EL SALVADOR,2021-06-17,707,198615,316
1,CHEDRAUI (APPRIZA),MEXICO,2021-06-17,32,17128,12
2,24XORO,MEXICO,2023-07-03,53,37002,1
3,BANCO DE GUAYAQUIL,ECUADOR,2022-08-16,105,54358,1
4,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,2021-11-12,696,391241,73


In [111]:
# Merge both df, keeping only daily_check data related to forecast
df_forecast = pd.merge(df_daily_check, final_df, on=['date','payer','country'], how='inner') 

In [112]:
df_forecast['date'].nunique()

49

In [113]:
df_forecast = df_forecast.loc[df_forecast.amount!=0] # To process

In [114]:
df_forecast["pe"] = abs((df_forecast['amount'] - df_forecast['mean']) / df_forecast['amount'])

In [115]:
df_forecast.pe.min(), df_forecast.pe.max()

(1.4803371099277962e-05, 596.8732956016264)

In [116]:
df_forecast = df_forecast.sort_values(['folder_id', 'country', 'payer','date'])

In [117]:
df_forecast.head(20)

,payer,country,date,tx,amount,coupon_count,folder_id,p10,p50,p90,mean,pe
1814,ARGENPER,ARGENTINA,2023-05-06,3,371,0,FORECAST_7D_05_05_2023,88,535,982,535,0
1118,ARGENPER,ARGENTINA,2023-05-07,3,235,0,FORECAST_7D_05_05_2023,-46,401,849,401,1
2278,ARGENPER,ARGENTINA,2023-05-08,1,100,0,FORECAST_7D_05_05_2023,192,639,1087,639,5
4447,ARGENPER,ARGENTINA,2023-05-09,3,394,0,FORECAST_7D_05_05_2023,164,611,1059,611,1
947,ARGENPER,ARGENTINA,2023-05-10,3,290,0,FORECAST_7D_05_05_2023,111,559,1007,559,1
1784,ARGENPER,ARGENTINA,2023-05-11,5,939,0,FORECAST_7D_05_05_2023,26,474,922,474,0
2566,ARGENPER,ARGENTINA,2023-05-12,6,1400,0,FORECAST_7D_05_05_2023,161,609,1057,609,1
3753,TRANSPAY,BANGLADESH,2023-05-08,1,399,1,FORECAST_7D_05_05_2023,-36,1186,2407,1186,2
268,TRANSFERTO - THUNES,BELGIUM,2023-05-10,1,46,0,FORECAST_7D_05_05_2023,-258,93,444,93,1
991,TRANSFERTO - THUNES,BELGIUM,2023-05-12,1,335,0,FORECAST_7D_05_05_2023,-258,93,444,93,1


In [118]:
df_forecast['payer'].unique()

array(['ARGENPER', 'TRANSPAY', 'TRANSFERTO - THUNES', 'BANCO BISA',
       'BANCO GANADERO (BOLIVIA)', 'BANCO DAYCOVAL', 'BANCO RENDIMENTO',
       'MSBB MONEY', 'KORI', 'NAFA', 'ZEEPAY', 'AFEX', 'BANCOLOMBIA',
       'DAVIVIENDA', 'GRUPO EXITO (TN)', 'BANCO BHD LEON',
       'CARIBE EXPRESS', 'BANCO DE GUAYAQUIL', 'BANCO DEL AUSTRO',
       'EASY PAGOS', 'ECUAGIROS', 'GLOBAL ENVIOS', 'ABANK (TN)',
       'BANCO AGRICOLA', 'BANCO DAVIVIENDA SALVADORENO (BTS)',
       'CREDOMATIC', 'CUSCA (CITI - RECIBA NETWORKS)',
       'FEDECACES (EL SALVADOR)', 'FEDECREDITO (RYT)', 'PROMERICA',
       'BAM - BANCO AGROMERCANTIL (UT)', 'BANCO INDUSTRIAL',
       'BANRURAL (RYT)', 'BANTRAB (UT)', 'ELEKTRA (BTS)',
       'G Y T CONTINENTAL', 'MICOOPE-FENACOAC (RED CHAPINA)',
       'PROMERICA (GUA)', 'AFRO INTERNACIONAL', 'UNITRANSFER',
       'BANCO ATLANTIDA', 'BANCO DE OCCIDENTE',
       'BANCO POPULAR HONDURAS (UT)', 'BANHCAFE', 'BANPAIS (RED CHAPINA)',
       'BANRURAL (HONDURAS)', 'COOPERATIVA CA

In [119]:
df_first = df_forecast.drop_duplicates(subset=['folder_id', 'country','payer'], keep='first')

In [120]:
df_first.groupby('folder_id')['pe'].mean()

folder_id
FORECAST_7D_05_05_2023   0
FORECAST_7D_10_09_2023   0
FORECAST_7D_20_03_2023   0
FORECAST_7D_20_12_2022   0
FORECAST_7D_25_11_2022   0
FORECAST_7D_29_12_2022   1
FORECAST_7D_30_06_2023   1
Name: pe, dtype: float64

In [121]:
# ELEKTRA MEXICO
df_first[df_first['payer'] == 'ELEKTRA (MEXICO)'].groupby('folder_id')['pe'].mean()

folder_id
FORECAST_7D_05_05_2023   0
FORECAST_7D_10_09_2023   0
FORECAST_7D_20_03_2023   0
FORECAST_7D_20_12_2022   0
FORECAST_7D_25_11_2022   0
FORECAST_7D_29_12_2022   0
FORECAST_7D_30_06_2023   0
Name: pe, dtype: float64

In [122]:
df_mex = df_forecast[df_forecast['country'] == 'MEXICO']

In [123]:
df_first_grouped = df_first.sort_values(['folder_id','country']).groupby(['folder_id', 'country'])['pe'].mean().reset_index()

In [127]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_first_grouped[df_first_grouped['country'] == 'ARGENTINA']

,folder_id,country,pe
0,FORECAST_7D_05_05_2023,ARGENTINA,0.44
39,FORECAST_7D_10_09_2023,ARGENTINA,0.29
84,FORECAST_7D_20_03_2023,ARGENTINA,0.10
119,FORECAST_7D_20_12_2022,ARGENTINA,0.25
153,FORECAST_7D_25_11_2022,ARGENTINA,0.36
187,FORECAST_7D_29_12_2022,ARGENTINA,0.52
221,FORECAST_7D_30_06_2023,ARGENTINA,0.46


In [126]:
pd.set_option('display.float_format', '{:.2f}'.format)
df_first_grouped[df_first_grouped['country'] == 'MEXICO']

,folder_id,country,pe
24,FORECAST_7D_05_05_2023,MEXICO,0.15
69,FORECAST_7D_10_09_2023,MEXICO,0.29
105,FORECAST_7D_20_03_2023,MEXICO,0.30
139,FORECAST_7D_20_12_2022,MEXICO,0.13
173,FORECAST_7D_25_11_2022,MEXICO,0.20
207,FORECAST_7D_29_12_2022,MEXICO,0.18
250,FORECAST_7D_30_06_2023,MEXICO,0.18


In [129]:
df_mex.head()

,payer,country,date,tx,amount,coupon_count,folder_id,p10,p50,p90,mean,pe
3050,AIRPAK,MEXICO,2023-05-06,282,107151.17,3,FORECAST_7D_05_05_2023,70957.97,100672.45,130386.93,100672.45,0.06
4367,AIRPAK,MEXICO,2023-05-07,396,160340.30,6,FORECAST_7D_05_05_2023,80323.75,114335.59,148347.43,114335.59,0.29
1535,AIRPAK,MEXICO,2023-05-08,412,130867.52,3,FORECAST_7D_05_05_2023,62449.74,89185.20,115920.67,89185.20,0.32
3632,AIRPAK,MEXICO,2023-05-09,396,122977.91,4,FORECAST_7D_05_05_2023,49373.96,70742.70,92111.44,70742.70,0.42
820,AIRPAK,MEXICO,2023-05-10,318,85153.38,7,FORECAST_7D_05_05_2023,46803.01,67278.42,87753.84,67278.42,0.21


In [132]:
df_mex_7d=df_forecast.loc[df_forecast.country=='MEXICO']
df_mex_7d.head()

,payer,country,date,tx,amount,coupon_count,folder_id,p10,p50,p90,mean,pe
3050,AIRPAK,MEXICO,2023-05-06,282,107151.17,3,FORECAST_7D_05_05_2023,70957.97,100672.45,130386.93,100672.45,0.06
4367,AIRPAK,MEXICO,2023-05-07,396,160340.30,6,FORECAST_7D_05_05_2023,80323.75,114335.59,148347.43,114335.59,0.29
1535,AIRPAK,MEXICO,2023-05-08,412,130867.52,3,FORECAST_7D_05_05_2023,62449.74,89185.20,115920.67,89185.20,0.32
3632,AIRPAK,MEXICO,2023-05-09,396,122977.91,4,FORECAST_7D_05_05_2023,49373.96,70742.70,92111.44,70742.70,0.42
820,AIRPAK,MEXICO,2023-05-10,318,85153.38,7,FORECAST_7D_05_05_2023,46803.01,67278.42,87753.84,67278.42,0.21


In [138]:
results_mex = df_mex_7d.groupby(['country', 'date']).agg({'amount': 'sum', 'mean': 'sum'}).reset_index()


In [139]:
results_mex.head()

,country,date,amount,mean
0,MEXICO,2022-11-26,16392615.77,19134014.07
1,MEXICO,2022-11-27,15367715.39,18280479.36
2,MEXICO,2022-11-28,14468737.03,15052783.70
3,MEXICO,2022-11-29,12165362.60,12484108.97
4,MEXICO,2022-11-30,11639041.43,11576455.73


In [149]:
results_mex['date'] = pd.to_datetime(results_mex['date'])

In [150]:
results_mex['forecast_error']= results_mex.amount - results_mex['mean']


In [161]:
results_mex.head(2)

,country,date,amount,mean,forecast_error,abs_forecast_error
0,MEXICO,2022-11-26,16392615.77,19134014.07,-2741398.31,2741398.31
1,MEXICO,2022-11-27,15367715.39,18280479.36,-2912763.97,2912763.97


In [155]:
results_mex['abs_forecast_error'] = results_mex['forecast_error'].abs()

In [160]:
results_mex.head(15)

,country,date,amount,mean,forecast_error,abs_forecast_error
0,MEXICO,2022-11-26,16392615.77,19134014.07,-2741398.31,2741398.31
1,MEXICO,2022-11-27,15367715.39,18280479.36,-2912763.97,2912763.97
2,MEXICO,2022-11-28,14468737.03,15052783.70,-584046.67,584046.67
3,MEXICO,2022-11-29,12165362.60,12484108.97,-318746.37,318746.37
4,MEXICO,2022-11-30,11639041.43,11576455.73,62585.69,62585.69
5,MEXICO,2022-12-01,11656846.26,11953925.20,-297078.94,297078.94
6,MEXICO,2022-12-02,17738682.70,17442805.64,295877.06,295877.06
7,MEXICO,2022-12-21,14385666.55,13802305.40,583361.15,583361.15
8,MEXICO,2022-12-22,16016125.68,14175883.17,1840242.51,1840242.51
9,MEXICO,2022-12-23,22185561.56,19998555.54,2187006.02,2187006.02


In [158]:
results_mex.abs_forecast_error.mean()

2424944.9285719595

In [159]:
results_mex.amount.mean()

15958012.678420408

In [162]:
results_mex.abs_forecast_error.sum()

118822301.50002602

In [163]:
results_mex.amount.sum()

781942621.2426

In [164]:
print(results_mex.abs_forecast_error.sum()/results_mex.amount.sum())

0.15195782691983617


In [ ]:
###SIN DATOS DE 2022##

In [168]:
results_mex_2023=results_mex.loc[results_mex.date>='2023-01-01']

In [169]:
results_mex_2023.head()

,country,date,amount,mean,forecast_error,abs_forecast_error
16,MEXICO,2023-01-01,4004948.21,12035763.95,-8030815.74,8030815.74
17,MEXICO,2023-01-02,15195059.45,13840919.03,1354140.42,1354140.42
18,MEXICO,2023-01-03,12807895.02,12005806.89,802088.13,802088.13
19,MEXICO,2023-01-04,11630571.28,11618820.64,11750.64,11750.64
20,MEXICO,2023-01-05,10658682.47,12049546.50,-1390864.03,1390864.03


In [170]:
results_mex_2023.abs_forecast_error.mean()

2470668.787897782

In [171]:
results_mex_2023.amount.mean()

16791953.8284

In [ ]:
results_mex_2023.abs_forecast_error.sum()

In [172]:
results_mex_2023.amount.sum()

554134476.3372

In [173]:
print(results_mex_2023.abs_forecast_error.sum()/results_mex_2023.amount.sum())

0.1471340865479974
